## 计算一个专业各个课程的关联度,然后存储起来

In [2]:
import requests
import jutil
import time
import json
import itertools
import math
import threading

In [3]:
def dump_obj(path,obj):
    '''先清空再保存对象到文件'''
    with open(path,'w') as f:
        json.dump(obj,f,ensure_ascii=False)

In [32]:
class CalRelation(threading.Thread):
    def __init__(self,speciality_code,grade='2015',temp_file_dir='/tmp/'):
        threading.Thread.__init__(self)
        self.speciality_code = speciality_code
        self.grade=grade
        self.temp_file_dir = temp_file_dir
        self.course_records = self.get_course_code()
        self.beginIndex = 0
        self.endIndex = len(self.course_records)
        self.isRun = True # 控制开关
        self.single_links = []
    
    def run(self):
        '''
        先读取code记录,取出计算了多少
        再开始计算links,中途保存结果
        '''
        while self.isRun:
            self.get_single_links()
        

    def stop(self):
        '''停止本次计算,保存结果,清理垃圾'''
        self.isRun = False
        
        print('已停止')
    
    def get_course_code(self):
        '''如果本地没有,则从数据库查出课程名和代码,然后组合成一对对的返回'''
        t_path = self.temp_file_dir+self.speciality_code+ '_course_records.txt'
        try:
            with open(t_path,'r') as f:
                c = json.load(f)
                return c
        except:
            sql = "select DISTINCT(course_name),course_code from view_stu_course_mark where speciality_code='%s' and course_type='必'  \
            and grade='%s'" %(self.speciality_code,self.grade)
            df = jutil.load_pd_df(sql)
            d_records = df.to_dict('records')
            records = []
            n = len(d_records)
            for i in range(n-1):
                for j in range(i+1,n):
                    t = (d_records[i],d_records[j])
                    records.append(t)
            dump_obj(t_path,records)
        return records
    
    def cal_relation(self,code1,code2):
        '''计算相似度调用'''
        url2 = 'http://localhost:8082/course/relation/'
        url = url2+code1+'/'+code2
        re = requests.get(url)
        return re.json()['data']
    
    def drop_nan(self,num):
        '''如果非数字,则返回0'''
        return 0 if math.isnan(num) else num/100

    def get_single_links(self):
        '''计算所有课程的两两关系,复杂度为n(n-1)/2'''
        batch_link = [] # 辅助写入文件的
        begin_time = time.time()
        for i in range(self.beginIndex,self.endIndex):
            if not self.isRun:
                break
            code1 = self.course_records[i][0]['course_code']
            code2 = self.course_records[i][1]['course_code']
            result = self.cal_relation(code1,code2)
            link = {}
            link['source'] = self.course_records[i][0]['course_name']
            link['target'] = self.course_records[i][1]['course_name']
            link['prob'] = self.drop_nan(float(result['prob']))
            link['bg_prob'] = self.drop_nan(float(result['bg_prob']))
            link['total'] = result['total']
            self.single_links.append(link)
            batch_link.append(link)
            # 周期性地写入文件,保存结果,因为计算量很大,需要很长时间
            if len(batch_link) > 30 or (i==self.endIndex-1):
                with open(self.temp_file_dir+self.speciality_code+ '_single_links.txt','a') as f:
                    for x in batch_link:
                        json.dump(x,f)
                        f.write('\n')
                    end_time = time.time()
#                     print('写入%d条信息,耗时%.2f 秒' % (len(batch_link),end_time-begin_time))
                    batch_link.clear()
                    begin_time = end_time
            self.beginIndex = i
        
    

## 测试

In [36]:
cal = CalRelation(speciality_code='0501',temp_file_dir='./')

In [37]:
cal.start()
# cal.join()

In [38]:
cal.stop()

已停止


In [4]:
def cal_relation(code1,code2):
    '''计算相似度调用'''
    url2 = 'http://localhost:8082/course/relation/'
    url = url2+code1+'/'+code2
    re = requests.get(url)
    return re.json()['data']

In [5]:
def drop_nan(num):
    '''如果非数字,则返回0'''
    return 0 if math.isnan(num) else num/100

In [6]:
def get_single_links(course_records):
    '''计算所有课程的两两关系,复杂度为n(n-1)/2'''
    single_links = []
    batch_link = [] # 辅助写入文件的
    begin_time = time.time()
    n = len(course_records)
    for i in range(n-1):
        for j in range(i+1,n):
            code1 = course_records[i]['course_code']
            code2 = course_records[j]['course_code']
            result = cal_relation(code1,code2)
            link = {}
            link['source'] = course_records[i]['course_name']
            link['target'] = course_records[j]['course_name']
            link['prob'] = drop_nan(float(result['prob']))
            link['bg_prob'] = drop_nan(float(result['bg_prob']))
            link['total'] = result['total']
            single_links.append(link)
            batch_link.append(link)
            # 周期性地写入文件,保存结果,因为计算量很大,需要很长时间
            if len(batch_link) > 30 or (i==n-2 and j==n-1):
                with open('single_links.txt','a') as f:
                    for x in batch_link:
                        json.dump(x,f)
                        f.write('\n')
                    end_time = time.time()
                    print('写入%d条信息,耗时%.2f 秒' % (len(batch_link),end_time-begin_time))
                    batch_link.clear()
                    begin_time = end_time
    return single_links

## 测试get_single_links

In [64]:
short_dict = course_records[:10]
single_links = get_single_links(short_dict)
single_links[:5]

写入0条信息,耗时17.04 秒
写入0条信息,耗时7.97 秒


[{'bg_prob': 0.0,
  'prob': 0.0,
  'source': '工程制图',
  'target': '综合运输概论',
  'total': 0},
 {'bg_prob': 0.0, 'prob': 0.0, 'source': '工程制图', 'target': '物流导论', 'total': 0},
 {'bg_prob': 0.0,
  'prob': 0.0,
  'source': '工程制图',
  'target': '管理信息系统',
  'total': 0},
 {'bg_prob': 0.0, 'prob': 0.0, 'source': '工程制图', 'target': '网络金融', 'total': 0},
 {'bg_prob': 0.2667,
  'prob': 0.5581,
  'source': '工程制图',
  'target': '高等数学BⅠ',
  'total': 86}]

## 生成nodes和links数据

In [7]:
def add_one_node_record(nodeRecord,name,prob):
    if name in nodeRecord:
        nodeRecord[name]['edgeCount'] += 1
        nodeRecord[name]['probSum'] += prob
    else :
        record = {}
        record['name'] = name
        record['edgeCount'] = 1
        record['probSum'] = prob
        nodeRecord[name] = record

In [8]:
def get_nodes_and_links(single_links):
    '''根据单个的课程间概率生成echarts展示的数据结构
    包括nodes和links
    nodes单个的结构如下:
    {
        category: 0,
        name: '高等数学',
        value: 10,
        symbolSize: 50 //圆圈的大小,一般大于30,这里我用与之相连的课程数和他们的概率来计算
    }
    links的单个结构如下:
    { source: '高等数学', target: '大学物理' }
    '''
    nodes = []
    nodeRecord = {} # 帮助计算node,存储的结构{name:{edgeCount:0,probSum:0.0}}
    links = []
    for slink in single_links:
        # 舍弃掉那些人数少的例子
        if slink['total'] > 30:
            link = {}
            link['source'] = slink['source']
            link['target'] = slink['target']
            links.append(link)
            add_one_node_record(nodeRecord,slink['source'],slink['prob'])
            add_one_node_record(nodeRecord,slink['target'],slink['prob'])
    # 计算nodes
#     print(nodeRecord)
    for name,v in nodeRecord.items():
        node = {}
        node['category'] = 0
        node['name'] = name
        node['symbolSize'] = v['edgeCount']*2 + v['probSum']*3
        nodes.append(node)
    return links,nodes
    

## 测试生成的nodes和links数据

In [29]:
def read_single_links():
    single_links = []
    with open('single_links.txt','r') as f:
        l = f.readlines()
        for x in l:
            d = json.loads(x)
            single_links.append(d)
    return single_links

In [51]:
links,nodes = get_nodes_and_links(single_links)

{'工程制图': {'name': '工程制图', 'edgeCount': 8, 'probSum': 499.69000000000005}, '高等数学BⅠ': {'name': '高等数学BⅠ', 'edgeCount': 3, 'probSum': 130.91}, '大学物理BI': {'name': '大学物理BI', 'edgeCount': 2, 'probSum': 145.82999999999998}, '测量学': {'name': '测量学', 'edgeCount': 1, 'probSum': 42.34}, '线性代数B': {'name': '线性代数B', 'edgeCount': 2, 'probSum': 106.80000000000001}, '军事技能训练': {'name': '军事技能训练', 'edgeCount': 2, 'probSum': 169.79}, '英语Ⅰ': {'name': '英语Ⅰ', 'edgeCount': 2, 'probSum': 147.97}, '高等数学BⅡ': {'name': '高等数学BⅡ', 'edgeCount': 2, 'probSum': 134.72}, '普通测量实验': {'name': '普通测量实验', 'edgeCount': 1, 'probSum': 69.23}, '综合运输概论': {'name': '综合运输概论', 'edgeCount': 13, 'probSum': nan}, '物流导论': {'name': '物流导论', 'edgeCount': 2, 'probSum': 103.49000000000001}, '分析化学': {'name': '分析化学', 'edgeCount': 1, 'probSum': 58.54}, '物流学': {'name': '物流学', 'edgeCount': 1, 'probSum': 74.7}, '英语综合能力MⅠ': {'name': '英语综合能力MⅠ', 'edgeCount': 1, 'probSum': 93.55}, '计算机程序设计基础A': {'name': '计算机程序设计基础A', 'edgeCount': 1, 'probSum': 68.97}, '无机化学

In [44]:
links

[{'source': '工程制图', 'target': '高等数学BⅠ'},
 {'source': '工程制图', 'target': '大学物理BI'},
 {'source': '工程制图', 'target': '测量学'},
 {'source': '工程制图', 'target': '线性代数B'},
 {'source': '工程制图', 'target': '军事技能训练'},
 {'source': '工程制图', 'target': '英语Ⅰ'},
 {'source': '工程制图', 'target': '高等数学BⅡ'},
 {'source': '工程制图', 'target': '普通测量实验'},
 {'source': '综合运输概论', 'target': '物流导论'},
 {'source': '综合运输概论', 'target': '高等数学BⅠ'},
 {'source': '综合运输概论', 'target': '大学物理BI'},
 {'source': '综合运输概论', 'target': '分析化学'},
 {'source': '综合运输概论', 'target': '线性代数B'},
 {'source': '综合运输概论', 'target': '军事技能训练'},
 {'source': '综合运输概论', 'target': '物流学'},
 {'source': '综合运输概论', 'target': '英语综合能力MⅠ'},
 {'source': '综合运输概论', 'target': '英语Ⅰ'},
 {'source': '综合运输概论', 'target': '高等数学BⅡ'},
 {'source': '综合运输概论', 'target': '计算机程序设计基础A'},
 {'source': '综合运输概论', 'target': '无机化学实验'},
 {'source': '综合运输概论', 'target': '管理学原理'},
 {'source': '物流导论', 'target': '高等数学BⅠ'}]

In [68]:
nodes

[{'category': 0, 'name': '工程制图', 'symbolSize': 68.9845},
 {'category': 0, 'name': '高等数学BⅠ', 'symbolSize': 35.5455},
 {'category': 0, 'name': '大学物理BI', 'symbolSize': 33.2915},
 {'category': 0, 'name': '测量学', 'symbolSize': 25.117},
 {'category': 0, 'name': '线性代数B', 'symbolSize': 31.34},
 {'category': 0, 'name': '军事技能训练', 'symbolSize': 34.4895},
 {'category': 0, 'name': '英语Ⅰ', 'symbolSize': 33.3985},
 {'category': 0, 'name': '高等数学BⅡ', 'symbolSize': 32.736000000000004},
 {'category': 0, 'name': '普通测量实验', 'symbolSize': 26.4615},
 {'category': 0, 'name': '综合运输概论', 'symbolSize': nan},
 {'category': 0, 'name': '物流导论', 'symbolSize': 31.174500000000002},
 {'category': 0, 'name': '分析化学', 'symbolSize': 25.927},
 {'category': 0, 'name': '物流学', 'symbolSize': 26.735},
 {'category': 0, 'name': '英语综合能力MⅠ', 'symbolSize': 27.677500000000002},
 {'category': 0, 'name': '计算机程序设计基础A', 'symbolSize': 26.4485},
 {'category': 0, 'name': '无机化学实验', 'symbolSize': nan},
 {'category': 0, 'name': '管理学原理', 'symbolSize'

In [32]:
def dump_obj(path,obj):
    '''先清空再保存对象到文件'''
    open(path,'w').close() # 先清空
    with open(path,'w') as f:
        json.dump(obj,f,ensure_ascii=False)

In [33]:
single_links = read_single_links()
links,nodes = get_nodes_and_links(single_links)
print('已生成nodes和links结构')
# 存到文件里
dump_obj("links.txt",links)
dump_obj("nodes.txt",nodes)
print('已存储在文件里')

已生成nodes和links结构
已存储在文件里


## 修正圆圈的大小

In [38]:
# 使最大不超过30
def correct_simbol_size(path):
    nodes = None
    with open('nodes.txt','r') as f:
        nodes = json.load(f)
#     print(nodes)
    # 找出最大的
    max_size = 0
    for node in nodes:
        if node['symbolSize']>max_size:
            max_size = node['symbolSize']
    scale = max_size/30
    nodes_now = []
    for node in nodes:
        node['symbolSize'] /= scale
        # 舍去小的
        if node['symbolSize'] > 1:
            nodes_now.append(node)
    dump_obj(path,nodes_now)

In [39]:
correct_simbol_size('new_nondes.txt')

## 组合上述方法

In [26]:
def final_cal_relation(speciality_code,grade,store_links_path='links.txt',store_nodes_path='nodes.txt'):
    course_records = get_course_code(speciality_code,grade)
    print('已获取该专业课程')
    single_links = get_single_links(course_records)
    print('完成所有课程关系的计算')
    links,nodes = get_nodes_and_links(single_links)
    print('已生成nodes和links结构')
    # 存到文件里
    dump_obj(store_links_path,links)
    dump_obj(store_nodes_path,nodes)
    print('已存储在文件里')
    # TODO 记录所有时间
    # 改成多线程下载

In [28]:
#final_cal_relation('0501','2015')
# 127.0.0.1 - - [18/Mar/2018 00:49:00] "GET /course/relation/8010220/7001152 HTTP/1.1" 500 

## 改成多线程

In [1]:
import threading

In [ ]:
class CalRelationThread(threading.Thread):
    '''计算一次专业的线程'''
    def __init__(self,name):
        threading.Thread.__init__(self)
        self.name = name
    
    def run(self):
        pass
    
    

# 部署方案

1. 修改代码,可以存储中间结果,这样随时可中断
2. 多线程计算,只考虑按专业来分,一个专业一个线程
3. 界面可控制整个过程,将中间结果存入文件